<H2>Detecting nodes within a distance that may occur due to unjoined mesh</H2> 

In [3]:
import pandas as pd
import numpy as np
import itertools   
import time
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.0")
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")
db = lusas.database()

Define the distance below which we want to detect adjacent nodes

In [4]:
TOLERANCE = 0.1

<H3>Option 1, using itertools to generate the necessary comparison combinations</H3>

In [ ]:
# Get all the nodes in the model
nodes = lusas.database().getObjects("Nodes")

if len(nodes) > 100:
    assert False, "This will take a very long time"

start = time.time()
no_comparisons = 0

# Use itertools to generate the comparison of each node with each other node
# No of comparisons will be n? where the ? indicates sumation of all integers fron n to 0
for a, b in itertools.combinations(nodes, 2):
    no_comparisons+=1
    delta = np.sqrt( (a.getX() - b.getX())**2 + (a.getY() - b.getY())**2 + (a.getZ() - b.getZ())**2 )
    if delta < TOLERANCE:
        print(f"Nodes {a.getID()} and {b.getID()} are within {delta:.3f}")

print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")
